In [1]:
'''
Step 1
Convolution - Take a big matrix of pixels and simply it down to smaller matrix.
    This is called a feature map. Big image and matrix of pixels, take a rolling
    window of 3x3 and sum pixels and then it creates a feature map and in turn
    a smaller matrix.
    
    Take many feature maps by trying to identify different features.
    This makes lots of convolutional layers, then we can work out what features
    make up this image.
    
     Convulations/ this process is also used for bluring and sharpening images.
     As we apply different convulation matrixes to the original image and it
     will change how the image looks.
     
     Google: Feature detectors.

Step 2
Max Pooling/Down smapling - Attempts to find the generic features in images to
    identify features.
    Otherwise, things like lighting, shaddow and angle of head would throw off 
    result that we put into the network.
    Similar process to convluation but operates on feature maps. 
    Take the max value to avoid image distortion/lighting/shaddow (mroe generic).
    
    Google: scs.ryerson.ca/~aharley/vis/conv/flat.html
    
Step 3
Flattening - Take the pooled feature map and turn into a 1xN array. Vector of inputs.

Step 4 
Full Connection - Use the flattening arrays as input to the network.


Softmax & Cross-Entropy - Bring output values between 0-1 to get a probability.
'''

In [21]:
# Import librarys
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

IMG_SIZE = 64

In [3]:
# Initialising the CNN
model = Sequential()

In [4]:
# Step 1 - Convolution
# help(Convolution2D)
model.add(
    Convolution2D(
        filters=32, # how many feature maps we want
        kernel_size=3, # size of rolling window
        input_shape=(IMG_SIZE, IMG_SIZE, 3), # tje shape and RGB of the images
        activation='relu', # activation function
    )
)

W0802 16:57:50.967326 11152 deprecation.py:506] From h:\code\machine-learning\machine-learning-playground\.venv\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
# Step 2 - Pooling
# help(MaxPooling2D)
model.add(
    MaxPooling2D(
        pool_size=(2, 2), # half on x & y the feature maps to pooling martix
    )
)

In [6]:
# Adding a second convolution layer for increased accuracy
model.add(
    Convolution2D(
        filters=32, # how many feature maps we want
        kernel_size=3, # size of rolling window
        activation='relu', # activation function
    )
)
model.add(
    MaxPooling2D(
        pool_size=(2, 2), # half on x & y the feature maps to pooling martix
    )
)

In [7]:
# Step 3 - Flattening
# help(Flatten
model.add(Flatten())

In [8]:
# Step 4 - Full Connection
# help(Dense)
model.add(
    Dense(
        128, # number of neurons (~average between no inputs and outcomes)
        activation='relu', # activation function
    )
)

In [9]:
# Output layer
model.add(
    Dense(
        1, # number of neurons (~average between no inputs and outcomes)
        activation='sigmoid', # activation function - binary func
    )
)

In [10]:
# Compiling the CNN
model.compile(
    optimizer='adam', # how to optimse the netwrok
    loss='binary_crossentropy', # loss function
    metrics=['accuracy'] # what we're interested in
)

W0802 16:57:51.093325 11152 deprecation.py:323] From h:\code\machine-learning\machine-learning-playground\.venv\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
# Fitting the CNN to the images
# changes the images to stop overfitting - like stretching, rotating and zooming images. 
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


root = 'H:/Code/machine-learning/Machine-Learning-Playground/1.0 - Example Data/Part 8 - Deep Learning/Convolutional_Neural_Networks/'
training_set = train_datagen.flow_from_directory(
        root + 'dataset/training_set',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=25,
        class_mode='binary'
)

test_set = test_datagen.flow_from_directory(
        root + 'dataset/training_set',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=25,
        class_mode='binary'
)

# THIS TAKES LIKE >2hrs!!!
model.fit_generator(
        training_set,
        steps_per_epoch=8000,
        epochs=25,
        validation_data=test_set,
        validation_steps=2000,
)

Found 8000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Epoch 1/25
8000/8000 [==============================] - 358s 45ms/step - loss: 0.3946 - acc: 0.8127 - val_loss: 0.1624 - val_acc: 0.9375
Epoch 2/25
8000/8000 [==============================] - 360s 45ms/step - loss: 0.1496 - acc: 0.9400 - val_loss: 0.0407 - val_acc: 0.9881
Epoch 3/25
8000/8000 [==============================] - 367s 46ms/step - loss: 0.0750 - acc: 0.9728 - val_loss: 0.0295 - val_acc: 0.9894
Epoch 4/25
8000/8000 [==============================] - 353s 44ms/step - loss: 0.0506 - acc: 0.9822 - val_loss: 0.0151 - val_acc: 0.9942
Epoch 5/25
8000/8000 [==============================] - 351s 44ms/step - loss: 0.0390 - acc: 0.9863 - val_loss: 0.0263 - val_acc: 0.9890
Epoch 6/25
8000/8000 [==============================] - 350s 44ms/step - loss: 0.0326 - acc: 0.9887 - val_loss: 0.0333 - val_acc: 0.9889
Epoch 7/25
8000/8000 [==============================] - 351s 44ms/step - loss: 0.0292 - acc: 0

In [22]:
# val_acc: 0.9971